This blog post describes a possible use case of the `fumbbl_replays` package.

To install the `fumbbl_replays` python package, follow the instructions on Github.

In [ ]:
#%pip install .

# Application: Plotting defensive tournament setups

Here the idea is to visualize defensive setups in a tournament setting.
I am currently playing amazons in our local league. Next is to play them in a tournament.
So lets think about setups with a EB/WC style Amazon roster, and learn from the best.

To learn from the best, we need to find high stakes tournament games with EB/WC rosters on FUMBBL.
The Tilean Team Cup checks these boxes. It was an online NAF tournament in 2023 on FUMBBL that used the World Cup ruleset.
https://member.thenaf.net/index.php?module=NAF&type=tournaments&func=view&id=7495

The tournament was even blogged about by NAF tournament director Stimme, who wrote:

https://www.thenaf.net/2023/05/tournament-director-blog-may-2023/

*Among the individual coaches, Siggi stood out with his Amazons, earning the title of best coach with a flawless record of six wins. Kurjo’s Orcs secured second place, while helborg’s Dark Elves came in third, both with five wins and a draw.* 


In [ ]:
import fumbbl_replays as fb
import pandas as pd

# list of EB/WC amazon teams by strong coaches
team_ids = [1117702, # siggi
1210795, # jleav
1212630, # sann
1215544, # yuri 
1216461, # bazakine
1210794] #tripleskull

match_ids = []

for team_id in team_ids:

    race_name = fb.fetch_team(team_id)['roster']['name']

    json_matches = fb.fetch_team_matches(team_id)

    for i in range(len(json_matches)):
        match_ids.append(json_matches[i]['id'])

len(match_ids)

# The Amazon roster in the WC2023 ruleset

First lets have a look at the roster. Siggi went with a roster of 13 players, 2 rerolls (plus a leader reroll), apothecary AND there was even money left for a 100K bribe to be used together with Dirty Player.

In [ ]:
my_replay = fb.fetch_replay(match_id = min(match_ids))
players = fb.extract_players_from_replay(my_replay)
rosters = fb.extract_rosters_from_replay(my_replay) 

(rosters
 .query('race == @race_name')
 .filter(['race' , 'short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury', 'reRolls', 'apothecaries'])
)


JackassRampant wrote an awesome series of articles discussing defensive setups.
https://fumbbl.com/p/notes?op=view&id=9773

# Plotting the defensive setups used

In [ ]:
do_refresh = False

replay_ids = []
race_defense = []
race_offense = []

for match_id in match_ids:
    print(".", end = '')
    # fetch and parse replay (positions contains board state at kick-off)
    match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id)
    # create plots
    plot = fb.create_defense_plot(replay_id, match_id, positions, receiving_team, metadata, refresh = do_refresh) 
    plot = fb.create_offense_plot(replay_id, match_id, positions, receiving_team, metadata, refresh = do_refresh) 

    replay_ids.append(int(replay_id))
    race_defense.append(metadata[4])
    race_offense.append(metadata[5])

df_replays = pd.DataFrame( {"matchId": match_ids,
                            "replayId": replay_ids,
                            "raceOffense": race_offense,
                            "raceDefense": race_defense
                            })

df_replays

In [ ]:
from PIL import Image
import os

for i in range(len(df_replays)):
    if df_replays.iloc[i]['raceDefense'] == race_name:
        dirname = "kickoff_pngs/" + df_replays.iloc[i]['raceDefense'] + "/"
        dirname = dirname.lower()
        dirname = dirname.replace(' ', '_')
        fname = dirname + str(df_replays.iloc[i]['replayId']) + "_" + \
            str(df_replays.iloc[i]['matchId']) + "_kickoff_lower_defense.png" 
        print(fname)
        if os.path.isfile(fname):
            display(Image.open(fname))
    

Apparently, coach siggi used an asymmetric setup against his Skaven opponent, with the weakest players on the line as cannon fodder, and positional players with good defensive skills such as block and wrestle further back. The dirty player lineman, Leader Thrower and Guard blocker are safe on row four.